In [1]:
from __future__ import print_function, division
import torch
import torch.nn as nn
from torchvision import models
import os,sys
import matplotlib.pyplot as plt
import numpy as np
from utils import train,test
import pandas as pd
import glob
from skimage import io

os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [2]:
def load_torch_model(model_ft,model_path,device):
    for param in model_ft.parameters():
        param.requires_grad = False    
    param_update=[]
    for name,param in model_ft.layer3.named_parameters():
        param.requires_grad = True
        param_update.append(param)
    for name,param in model_ft.layer4.named_parameters():
        param.requires_grad = True
        param_update.append(param)
    for name,param in model_ft.fc.named_parameters():
        param.requires_grad = True
        param_update.append(param)
    model_ft = nn.DataParallel(model_ft)
    
    model_wts = torch.load(model_path)
    model_ft.load_state_dict(model_wts)    
    model_ft = model_ft.to(device)
    return model_ft

In [3]:
if __name__ == '__main__':
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")        
    batch_size = 256
    data_dir_ori = {'train':'/cptjack/totem_disk/totem/M_MSI_MSS/normal/train',
                'val':'/cptjack/totem_disk/totem/M_MSI_MSS/normal/val'}
    
    model_save_path = '/cptjack/totem_disk/totem/kwong/CRC_DC_TRAIN/MIP'
    model_base_name = 'resnet_18'

    model_path = '/cptjack/totem_disk/totem/kwong/CRC_DC_TRAIN/MIL/resnet_18_2.pth'
    model_ft = models.resnet18(pretrained = False)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, 2)
    model_ft = load_torch_model(model_ft,model_path,device)

In [5]:
    train_epoch = train.Epoch(
            model_ft, 
            data_dir_ori, 
            1, 
            model_save_path,
            model_base_name,
            batch_size
            )
    
    model_wts,train_loss,train_acc,val_loss,val_acc = train_epoch.run()

Epoch 0/29
----------
train: 100%|██████████| 1165/1165 [1:57:19<00:00,  6.15s/it, CrossEntropyLoss - 0.526, Balanced_acc - 0.8654, recall - 0.7586]  
train Loss: 0.5721 Acc: 0.7202

val: 100%|██████████| 329/329 [23:35<00:00,  3.52s/it, CrossEntropyLoss - 0.4974, Balanced_acc - 0.7943, recall - 0.75]  
val Loss: 0.5262 Acc: 0.7346

Epoch early stop: 0/29
Epoch 1/29
----------
train: 100%|██████████| 1165/1165 [1:24:11<00:00,  3.20s/it, CrossEntropyLoss - 0.4768, Balanced_acc - 0.6795, recall - 0.6897]
train Loss: 0.5120 Acc: 0.7431

val: 100%|██████████| 329/329 [23:01<00:00,  3.51s/it, CrossEntropyLoss - 0.5336, Balanced_acc - 0.7197, recall - 0.6875]
val Loss: 0.5189 Acc: 0.7416

Epoch early stop: 1/29
Epoch 2/29
----------
train: 100%|██████████| 1165/1165 [1:23:59<00:00,  3.21s/it, CrossEntropyLoss - 0.4372, Balanced_acc - 0.795, recall - 0.8276] 
train Loss: 0.4886 Acc: 0.7589

val: 100%|██████████| 329/329 [23:13<00:00,  3.45s/it, CrossEntropyLoss - 0.5862, Balanced_acc - 0.6978